In [2]:
pip install torch opencv-python pygame scipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import torch
import cv2
import time
import numpy as np
import os
import pygame  # For audio alerts

# # Initialize pygame for audio
# pygame.mixer.init()
# # Create a sound for alert (you can replace this with your own sound file)
# alert_sound_path = os.path.join(os.path.dirname(_file_), "alert.wav")
# # If the sound file doesn't exist, create a simple alarm
# if not os.path.exists(alert_sound_path):
#     try:
#         from scipy.io.wavfile import write as write_wav
#         import numpy as np
        
#         # Generate a simple beep sound
#         sample_rate = 44100
#         duration = 1  # seconds
#         frequency = 440  # Hz (A4)
#         t = np.linspace(0, duration, int(sample_rate * duration), False)
#         beep = np.sin(2 * np.pi * frequency * t) * 0.5
#         beep = (beep * 32767).astype(np.int16)
        
#         os.makedirs(os.path.dirname(alert_sound_path), exist_ok=True)
#         write_wav(alert_sound_path, sample_rate, beep)
#         print(f"Created alert sound at {alert_sound_path}")
#     except Exception as e:
#         print(f"Could not create alert sound: {e}")
#         alert_sound_path = None

# # Try to load the alert sound
# try:
#     if alert_sound_path and os.path.exists(alert_sound_path):
#         alert_sound = pygame.mixer.Sound(alert_sound_path)
#     else:
#         alert_sound = None
#         print("No alert sound available")
# except Exception as e:
#     print(f"Error loading sound: {e}")
#     alert_sound = None

# Load the trained YOLOv5 model
try:
    # Load custom model if available
    model = torch.hub.load('ultralytics/yolov5', 'custom', path="C:\\Users\\Saksham Gupta\\Driver Drowsiness\\yolov5\\runs\\train\\exp5\\weights\\best.pt", force_reload=True)
    
except Exception as e:
    print(f"Error loading custom model: {e}")
    print("Falling back to pre-trained YOLOv5s model")
    # Fallback to pre-trained model
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Set confidence threshold
model.conf = 0.5  # Adjust confidence threshold as needed

# Check for CUDA availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(f"Using device: {device}")

# Define class names (adjust based on your model's classes)
class_names = ['awake', 'drowsy']

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam")
    exit()

# Initialize variables for alerting
drowsy_start_time = None
# alert_threshold = 3  # Alert if drowsy for 3 seconds
# alert_active = False

print("Starting real-time drowsiness detection. Press 'q' to quit.")

try:
    while True:
        # Read a frame from the webcam
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
            
        # Start timing for processing rate calculation
        start = time.time()
        
        # Process the frame with the model
        results = model(frame)
        
        # Get all predictions
        predictions = results.pred[0]  # shape: [num_predictions, 6]
        
        current_state = "Unknown"
        confidence = 0
        
        if len(predictions) > 0:
            # Filter predictions for relevant classes only
            # Note: Your model might use different class indices
            # First determine what classes are actually present in the predictions
            present_classes = predictions[:, 5].unique().int().tolist()
            print(f"Classes detected: {present_classes}") if len(present_classes) > 0 else None
            
            # Try to find the right classes based on what we see
            # If your model was trained with classes 0 and 1, use those
            if 0 in present_classes or 1 in present_classes:
                filtered_preds = predictions[(predictions[:, 5] == 0) | (predictions[:, 5] == 1)]
            # If your model was trained with classes 15 and 16 (as in your original code)
            elif 15 in present_classes or 16 in present_classes:
                filtered_preds = predictions[(predictions[:, 5] == 15) | (predictions[:, 5] == 16)]
            else:
                # Just use all predictions if we can't identify specific classes
                filtered_preds = predictions
            
            if len(filtered_preds) > 0:
                # Get the prediction with the highest confidence
                best_pred = filtered_preds[torch.argmax(filtered_preds[:, 4])]
                class_idx = int(best_pred[5])
                confidence = float(best_pred[4])
                
                # Map class index to name
                # If we have a custom model with classes from 0 to len(class_names)-1
                if class_idx < len(class_names):
                    current_state = class_names[class_idx]
                # For models with different class indices
                elif class_idx == 15:
                    current_state = "awake"
                elif class_idx == 16:
                    current_state = "drowsy"
                else:
                    current_state = f"Class {class_idx}"
                
                # # Handle drowsiness alert logic
                # if current_state == "drowsy":
                #     if drowsy_start_time is None:
                #         drowsy_start_time = time.time()
                #     elif time.time() - drowsy_start_time > alert_threshold and not alert_active:
                #         # Play sound alert if available
                #         if alert_sound:
                #             alert_sound.play()
                #         else:
                #             print("\a")  # Terminal bell sound (basic alert)
                #         alert_active = True
                # else:
                #     drowsy_start_time = None
                #     alert_active = False
        
        # Calculate processing rate
        end = time.time()
        fps = 1 / (end - start)
        
        # Display the results on the frame
        frame_with_results = results.render()[0].copy()  # Create a copy to make it writable
        
        # Add more info to the frame
        cv2.putText(frame_with_results, f"State: {current_state} ({confidence:.2f})", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame_with_results, f"FPS: {fps:.2f}", 
                   (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        if drowsy_start_time is not None:
            drowsy_time = time.time() - drowsy_start_time
            cv2.putText(frame_with_results, f"Drowsy for: {drowsy_time:.1f}s", 
                       (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
            # # Visual alert if threshold exceeded
            # if drowsy_time > alert_threshold:
            #     # Draw red border to indicate alert
            #     h, w = frame_with_results.shape[:2]
            #     cv2.rectangle(frame_with_results, (0, 0), (w, h), (0, 0, 255), 20)
                
                # # Display warning text
                # cv2.putText(frame_with_results, "DROWSINESS ALERT!", 
                #            (w//2-200, h//2), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)
        
        # Display the frame
        cv2.imshow('Driver Drowsiness Detection', frame_with_results)
        
        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Release resources
    cap.release()
    cv2.destroyAllWindows()
    print("Drowsiness detection stopped")

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Saksham Gupta/.cache\torch\hub\master.zip
YOLOv5  2025-5-20 Python-3.12.0 torch-2.4.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 1782166 parameters, 0 gradients, 4.2 GFLOPs
Adding AutoShape... 


Using device: cpu
Starting real-time drowsiness detection. Press 'q' to quit.


C:\Users\Saksham Gupta/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Saksham Gupta/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Saksham Gupta/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Saksham Gupta/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Saksham Gupta/.cache\to

Drowsiness detection stopped


In [6]:
import torch
import cv2
import time
import numpy as np

# Load the trained YOLOv5 model
try:
    # Load custom model if available
    model = torch.hub.load('ultralytics/yolov5', 'custom', path="C:\\Users\\Saksham Gupta\\Driver Drowsiness\\yolov5\\runs\\train\\exp5\\weights\\best.pt", force_reload=True)
    
except Exception as e:
    print(f"Error loading custom model: {e}")
    print("Falling back to pre-trained YOLOv5s model")
    # Fallback to pre-trained model
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Set confidence threshold
model.conf = 0.5  # Adjust confidence threshold as needed

# Check for CUDA availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(f"Using device: {device}")

# Define class names (adjust based on your model's classes)
class_names = ['awake', 'drowsy']

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam")
    exit()

# Initialize variables for drowsiness tracking
drowsy_start_time = None
alert_threshold = 3  # Consider drowsy after 3 seconds

print("Starting real-time drowsiness detection. Press 'q' to quit.")

try:
    while True:
        # Read a frame from the webcam
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
            
        # Start timing for processing rate calculation
        start = time.time()
        
        # Process the frame with the model
        results = model(frame)
        
        # Get all predictions
        predictions = results.pred[0]  # shape: [num_predictions, 6]
        
        current_state = "Unknown"
        confidence = 0
        
        if len(predictions) > 0:
            # Filter predictions for relevant classes only
            # First determine what classes are actually present in the predictions
            present_classes = predictions[:, 5].unique().int().tolist()
            print(f"Classes detected: {present_classes}") if len(present_classes) > 0 else None
            
            # Try to find the right classes based on what we see
            # If your model was trained with classes 0 and 1, use those
            if 0 in present_classes or 1 in present_classes:
                filtered_preds = predictions[(predictions[:, 5] == 0) | (predictions[:, 5] == 1)]
            # If your model was trained with classes 15 and 16 (as in your original code)
            elif 15 in present_classes or 16 in present_classes:
                filtered_preds = predictions[(predictions[:, 5] == 15) | (predictions[:, 5] == 16)]
            else:
                # Just use all predictions if we can't identify specific classes
                filtered_preds = predictions
            
            if len(filtered_preds) > 0:
                # Get the prediction with the highest confidence
                best_pred = filtered_preds[torch.argmax(filtered_preds[:, 4])]
                class_idx = int(best_pred[5])
                confidence = float(best_pred[4])
                
                # Map class index to name
                # If we have a custom model with classes from 0 to len(class_names)-1
                if class_idx < len(class_names):
                    current_state = class_names[class_idx]
                # For models with different class indices
                elif class_idx == 15:
                    current_state = "awake"
                elif class_idx == 16:
                    current_state = "drowsy"
                else:
                    current_state = f"Class {class_idx}"
                
                # Handle drowsiness tracking logic
                if current_state == "drowsy":
                    if drowsy_start_time is None:
                        drowsy_start_time = time.time()
                else:
                    drowsy_start_time = None
        
        # Calculate processing rate
        end = time.time()
        fps = 1 / (end - start)
        
        # Display the results on the frame
        frame_with_results = results.render()[0].copy()  # Create a copy to make it writable
        
        # Add more info to the frame
        cv2.putText(frame_with_results, f"State: {current_state} ({confidence:.2f})", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame_with_results, f"FPS: {fps:.2f}", 
                   (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        if drowsy_start_time is not None:
            drowsy_time = time.time() - drowsy_start_time
            cv2.putText(frame_with_results, f"Drowsy for: {drowsy_time:.1f}s", 
                       (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
            # Visual indicator if threshold exceeded
            if drowsy_time > alert_threshold:
                # Draw red border to indicate drowsiness detected
                h, w = frame_with_results.shape[:2]
                cv2.rectangle(frame_with_results, (0, 0), (w, h), (0, 0, 255), 20)
                
                # Display warning text
                cv2.putText(frame_with_results, "DROWSINESS DETECTED", 
                           (w//2-200, h//2), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)
        
        # Display the frame
        cv2.imshow('Driver Drowsiness Detection', frame_with_results)
        
        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Release resources
    cap.release()
    cv2.destroyAllWindows()
    print("Drowsiness detection stopped")

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Saksham Gupta/.cache\torch\hub\master.zip
YOLOv5  2025-5-20 Python-3.12.0 torch-2.4.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 1782166 parameters, 0 gradients, 4.2 GFLOPs
Adding AutoShape... 


Using device: cpu
Starting real-time drowsiness detection. Press 'q' to quit.


C:\Users\Saksham Gupta/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Saksham Gupta/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Saksham Gupta/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Saksham Gupta/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Saksham Gupta/.cache\to

Drowsiness detection stopped
